# Convolutional Neural Network

### Importing the libraries

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.preprocessing import image

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [18]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [19]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [20]:
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8002 images belonging to 2 classes.


In [21]:
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2021 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [22]:
cnn = Sequential()

### Step 1 - Convolution

In [23]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [24]:
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [25]:
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [26]:
cnn.add(Flatten())

### Step 4 - Full Connection

In [27]:
cnn.add(Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [28]:
cnn.add(Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [29]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [30]:
cnn.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=25,
    validation_data=test_set,
    validation_steps=len(test_set)
)

Epoch 1/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 187s 450ms/step - accuracy: 0.5297 - loss: 0.7000 - val_accuracy: 0.6705 - val_loss: 0.6515
Epoch 2/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 111us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 161s 561ms/step - accuracy: 0.6580 - loss: 0.6219 - val_accuracy: 0.7180 - val_loss: 0.5611
Epoch 4/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 172s 445ms/step - accuracy: 0.7153 - loss: 0.5597 - val_accuracy: 0.7486 - val_loss: 0.5317
Epoch 6/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 114s 448ms/step - accuracy: 0.7329 - loss: 0.5255 - val_accuracy: 0.7595 - val_loss: 0.5244
Epoch 8/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 134s 441ms/step - accuracy: 0.7588 - loss: 0.4983 

In [35]:
test_loss, test_accuracy = cnn.evaluate(test_set)
print(f"Test Accuracy: {test_accuracy:.2f}")

64/64 ━━━━━━━━━━━━━━━━━━━━ 17s 272ms/step - accuracy: 0.7386 - loss: 0.5464
Test Accuracy: 0.74


## Part 4 - Making a single prediction

In [32]:
def predict_image(image_path):
    img = image.load_img(image_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale

    prediction = cnn.predict(img_array)
    if prediction[0][0] > 0.5:
        print("Prediction: Dog")
    else:
        print("Prediction: Cat")

In [33]:
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_1.jpg')
predict_image('/content/drive/MyDrive/dataset/single_prediction/cat_or_dog_2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Prediction: Dog
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: Dog


In [34]:
cnn.save('cat_dog_classifier.h5')